<a href="https://colab.research.google.com/github/pedrovg97/ProjetoFinalED2/blob/main/projeto_final_ED2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install networkx nltk matplotlib
%pip install transformers
%pip install lxml

Note: you may need to restart the kernel to use updated packages.



Etapa 1: Seleção e Preparação dos Textos

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, ne_chunk, word_tokenize
from nltk.tree import Tree
from nltk import download
import torch
from transformers import BertTokenizer, BertForTokenClassification
from tqdm.notebook import tqdm
import pandas as pd
import spacy
import networkx as nx
import re
from spacy.lang.en.stop_words import STOP_WORDS

# Carregue o modelo do spaCy para inglês
nlp = spacy.load("en_core_web_sm")
new_stop_words = ['said', 'they', 's', 'nt', 'said', 'told', 'home', 'email']
nlp.Defaults.stop_words.update(new_stop_words)

GRAU_MINIMO = 3
FREQUENCIA_MINIMA = 5

# Carregar o modelo pré-treinado e o tokenizador para BERT NER
modelo_ner = BertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
tokenizer_ner = BertTokenizer.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')


# Carregar o modelo pré-treinado e o tokenizador para BERT
modelo_pos_tagging = BertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Baixar recursos necessários do NLTK (faça isso na primeira execução)
download('punkt')
download('averaged_perceptron_tagger')
download('maxent_ne_chunker')
download('words')


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trai

True

In [ ]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.7/12.8 MB 21.8 MB/s eta 0:00:01
     -------- ------------------------------- 2.7/12.8 MB 28.5 MB/s eta 0:00:01
     ------------ --------------------------- 3.9/12.8 MB 31.0 MB/s eta 0:00:01
     ------------- -------------------------- 4.4/12.8 MB 25.4 MB/s eta 0:00:01
     ----------------- ---------------------- 5.4/12.8 MB 24.8 MB/s eta 0:00:01
     ------------------ --------------------- 6.0/12.8 MB 22.6 MB/s eta 0:00:01
     ------------------------ --------------- 7.9/12.8 MB 26.7 MB/s eta 0:00:01
     ------------------------- -------------- 8.1/12.8 MB 26.0 MB/s eta 0:00:01
     -------------------------------- ------ 10.6/12.8 MB 28.4 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 28.5 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 28.5 MB/s eta 0:00:01
     --------------------------------------- 12.

### Preparação do Dataset

In [ ]:
import os

# Montar o Google Drive
#drive.mount('/content/drive')

# Diretório contendo os arquivos .txt
diretorio = '/content/sample_data'

# Nome do arquivo de saída
arquivo_saida = 'dataset.txt'

# Lista para armazenar os conteúdos dos arquivos
conteudos = []

# Iterar pelos arquivos no diretório
for nome_arquivo in os.listdir(diretorio_drive):
    caminho_arquivo = os.path.join(diretorio_drive, nome_arquivo)

    # Verificar se é um arquivo .txt
    if os.path.isfile(caminho_arquivo) and nome_arquivo.endswith('.txt'):
        # Ler o conteúdo do arquivo e adicioná-lo à lista
        with open(caminho_arquivo, 'r', encoding='utf-8', errors='ignore') as arquivo:
            conteudo_arquivo = arquivo.read()
            conteudos.append(conteudo_arquivo)

# Juntar todos os conteúdos em uma única string
conteudo_final = '\n'.join(conteudos)

# Caminho para salvar o arquivo de saída no Google Drive
caminho_saida = os.path.join(diretorio_drive, arquivo_saida)

# Escrever o conteúdo final em um único arquivo no Google Drive
with open(caminho_saida, 'w', encoding='utf-8', errors='ignore') as arquivo_final:
    arquivo_final.write(conteudo_final)

print(f'Todos os arquivos .txt foram combinados em {caminho_saida}.')


Todos os arquivos .txt foram combinados em /content/sample_data/dataset.txt.


In [ ]:
import os
import pandas as pd

# Diretório contendo os arquivos .txt
diretorio = './Fake'

# Lista para armazenar os conteúdos dos arquivos
conteudos = []

# Iterar pelos arquivos no diretório
for nome_arquivo in os.listdir(diretorio):
    caminho_arquivo = os.path.join(diretorio, nome_arquivo)

    # Verificar se é um arquivo .txt
    if os.path.isfile(caminho_arquivo) and nome_arquivo.endswith('.txt'):
        # Ler o conteúdo do arquivo e adicionar à lista
        with open(caminho_arquivo, 'r', encoding='utf-8', errors='ignore') as arquivo:
            conteudo_arquivo = arquivo.read()
            conteudos.append(conteudo_arquivo)

# Criar um DataFrame
df_fake = pd.DataFrame({'Texto': conteudos, 'Label': 1})

# Salvar o DataFrame em um arquivo CSV, se desejar
df_fake.to_csv('df_fake.csv', index=False, encoding='utf-8')

# Visualizar o DataFrame
print(df_fake.head())


                                               Texto  Label
0  The Internet is buzzing today after white supr...      1
1  TRUMP TOWER  With his poll numbers among black...      1
2  Donald Trump is looking to assemble a strong t...      1
3  SHES SEXY, SMART, SOPHISTICATED, AND SHES INTO...      1
4  PHOENIX A.Z. (AP)  For months now, rumors have...      1


In [ ]:
import os
import pandas as pd

# Diretório contendo os arquivos .txt
diretorio = './Real'

# Lista para armazenar os conteúdos dos arquivos
conteudos = []

# Iterar pelos arquivos no diretório
for nome_arquivo in os.listdir(diretorio):
    caminho_arquivo = os.path.join(diretorio, nome_arquivo)

    # Verificar se é um arquivo .txt
    if os.path.isfile(caminho_arquivo) and nome_arquivo.endswith('.txt'):
        # Ler o conteúdo do arquivo e adicionar à lista
        with open(caminho_arquivo, 'r', encoding='utf-8', errors='ignore') as arquivo:
            conteudo_arquivo = arquivo.read()
            conteudos.append(conteudo_arquivo)

# Criar um DataFrame
df_real = pd.DataFrame({'Texto': conteudos, 'Label': 0})

# Salvar o DataFrame em um arquivo CSV, se desejar
df_real.to_csv('df_real.csv', index=False, encoding='utf-8')

# Visualizar o DataFrame
print(df_real.head())


                                               Texto  Label
0  The list of Republicans supporting Hillary Cli...      0
1  CLEVELAND  Avik Roy is a Republicans Republica...      0
2  All politicians bend the truth to fit their pu...      0
3  m cool with you removing me from your friends ...      0
4  DONT go anywhere near a mosque, I told my moth...      0


In [ ]:
df_concat = pd.concat([df_fake, df_real], axis=0, ignore_index=True)
df_concat.to_csv('df_concat')

### Pos Tagging

Etapa 2: Análise de PoS Tagging

In [ ]:
# Carregar um texto de exemplo (substitua pelo seu próprio conjunto de dados)
with open(r"C:\Users\larissa.silva\Downloads\Buzzfeed Political News Dataset-20231217T195058Z-001\Buzzfeed Political News Dataset\Fake\1_Fake.txt", 'r', encoding='utf-8', errors='ignore') as arquivo:
    conteudo = arquivo.read()

# Tokenização por sentença e por palavra
texto_tokenizado = word_tokenize(conteudo)


In [ ]:
from nltk import pos_tag

# Realizar PoS Tagging nas palavras tokenizadas
pos_tags = pos_tag(texto_tokenizado)
print("PoS Tags:", pos_tags)


PoS Tags: [('Hillary', 'NNP'), ('in', 'IN'), ('hot', 'JJ'), ('water', 'NN'), ('over', 'IN'), ('her', 'PRP$'), ('email', 'NN'), ('server', 'NN'), (',', ','), ('again', 'RB'), ('.', '.'), ('Sacramento', 'NNP'), (',', ','), ('CA', 'NNP'), ('Democratic', 'NNP'), ('nominee', 'RB'), ('Hillary', 'NNP'), ('Clinton', 'NNP'), ('is', 'VBZ'), ('in', 'IN'), ('hot', 'JJ'), ('water', 'NN'), ('again', 'RB'), ('after', 'IN'), ('nearly', 'RB'), ('5', 'CD'), ('million', 'CD'), ('uncounted', 'JJ'), ('California', 'NNP'), ('electronic', 'JJ'), ('ballots', 'NNS'), ('were', 'VBD'), ('found', 'VBN'), ('on', 'IN'), ('her', 'PRP$'), ('email', 'NN'), ('server', 'NN'), ('by', 'IN'), ('the', 'DT'), ('F.B.I', 'NNP'), ('.', '.'), ('The', 'DT'), ('majority', 'NN'), ('of', 'IN'), ('those', 'DT'), ('ballots', 'NNS'), ('cast', 'VBP'), ('were', 'VBD'), ('by', 'IN'), ('Bernie', 'NNP'), ('Sanders', 'NNP'), ('supporters', 'NNS'), ('.', '.'), ('The', 'DT'), ('election', 'NN'), ('commission', 'NN'), ('has', 'VBZ'), ('an', 'DT

In [ ]:
### não usar: gera muitos nós

import pandas as pd
import networkx as nx
import spacy
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import re

# Função para remover pontuações, números e espaços de uma string
def preprocess_text(text):
    # Remover pontuações e números
    text = re.sub(r'[^\w\s]', '', text)
    # Remover espaços extras
    text = ' '.join(text.split())
    return text

# Carregar o modelo spaCy para POS tagging
nlp_spacy = spacy.load("en_core_web_sm")

# Carregar o modelo BERT pré-treinado e o tokenizer
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Seu código para criar o grafo usando NetworkX
G = nx.Graph()

# Iterar sobre os textos e adicionar nós e arestas ao grafo
for texto in df_fake['Texto']:
    # Pré-processar o texto para remover pontuações, números e espaços
    texto = preprocess_text(texto)

    # Tokenizar o texto usando o tokenizer BERT
    tokens = tokenizer(texto, return_tensors="pt", truncation=True, padding=True)

    # Obter as previsões do modelo para a classificação de tokens
    with torch.no_grad():
        outputs = model(**tokens)

    # Obter as tags POS usando spaCy
    doc_spacy = nlp_spacy(texto)

    # Definir uma frequência mínima para co-ocorrência
    cooccurrence_threshold = 2

    # Adicionar nós e arestas com base nas previsões do modelo e tags POS do spaCy
    for i, (token, pos_tag_spacy) in enumerate(zip(doc_spacy, [token.pos_ for token in doc_spacy])):
        # Verificar se o índice está dentro dos limites do modelo BERT
        if i < outputs.logits.shape[1]:
            G.add_node(token.text, pos_tag=pos_tag_spacy, pos_tag_bert=outputs.logits[0][i].argmax().item())

            # Adicionar arestas apenas entre tokens que ocorrem com frequência mínima
            for j in range(i + 1, len(doc_spacy)):
                if doc_spacy[j].text != token.text and G.has_node(doc_spacy[j].text):
                    G.add_edge(token.text, doc_spacy[j].text)

    # Exportar o grafo para um arquivo GraphML
    nx.write_graphml(G, "grafo_spacy_torch_fake.graphml")


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Dentro do loop para processar o texto
def processar_texto(texto:str)->list[str]:
    """
        Pré-processa o texto de cada linha do DataFrame, passando as palavras para minúsculo,
        removendo números e pontuações, filtrando stop words e transformando o texto em tokens.

        Args:
            - texto (str): qual o conjunto de dados.

        Returns:
            - pos_tags(list[str]): lista de tokens a serem processados.
    """

    texto = texto.lower()
    # Remover pontuações e números
    texto = re.sub(r'[^\w\s]', '', texto)
    # Remover espaços extras
    texto = ' '.join(texto.split())

    # Usar spaCy para obter PoS tags
    doc = nlp(texto)

    # Filtrar stop words
    pos_tags = [(token.text, token.pos_) for token in doc if token.text not in STOP_WORDS]

    return pos_tags

def create_graph_POS(df:pd.DataFrame, news:str)->None:
    """
        Gera um arquivo .graphml contendo a rede de palavras presentes nas 10 primeiras
            notícias do dataset e suas respectivas classes POS.

        Args:
            - df (pd.DataFrame): DataFrame contendo uma coluna com o texto de cada notícia.
            - news (str): qual o conjunto de dados esperados para compor o nome do arquivo
                gerado.

        Returns:
            None.
    """
    # Criar um dicionário para contar a frequência dos tokens
    frequencia_tokens = {}
    for _, row in df.iterrows():
        for token, _ in row['PoS_Tags']:
            frequencia_tokens[token] = frequencia_tokens.get(token, 0) + 1

    G = nx.DiGraph()

    # Adicione arestas apenas se o token for diferente do outro token
    for _, row in df.iterrows():
        for token, pos in row['PoS_Tags']:
            if frequencia_tokens[token] >= FREQUENCIA_MINIMA:
                G.add_node(token, pos=pos)
                for other_token, other_pos in row['PoS_Tags']:
                    if (
                        token != other_token
                        and frequencia_tokens[other_token] >= FREQUENCIA_MINIMA
                        and not G.has_edge(token, other_token)
                    ):
                        G.add_edge(token, other_token)

    # Filtrar nós com base no grau usando in_degree() para grafos direcionados
    nodes_filtrados = [node for node, in_degree in G.in_degree() if in_degree >= GRAU_MINIMO]

    # Imprima os nós filtrados
    print("Número de Nós no Subgrafo Filtrado:", len(nodes_filtrados))
    print("Nós no Subgrafo Filtrado:", nodes_filtrados)

    # Criar subgrafo apenas com os nós filtrados
    G_subgrafo = G.subgraph(nodes_filtrados)

    graph_name = 'grafo_spacy_'+news+'_filtrado_POS'
    # Salvar o grafo em formato GraphML
    nx.write_graphml(G_subgrafo, graph_name+".graphml")


# Aplicar as funções aos DataFrames
df_fake_part = df_fake.copy()[0:10].drop(axis=1, columns=['Label'])
df_fake_part['PoS_Tags'] = df_fake_part['Texto'].apply(processar_texto)

df_real_part = df_real.copy()[0:10].drop(axis=1, columns=['Label'])
df_real_part['PoS_Tags'] = df_real_part['Texto'].apply(processar_texto)

create_graph_POS(df_fake_part, 'fake')
create_graph_POS(df_real_part, 'real')

Número de Nós no Subgrafo Filtrado: 70
Nós no Subgrafo Filtrado: ['white', 'presidential', 'donald', 'trump', 'like', 'asked', 'called', 'trumps', 'campaign', 'real', 'day', 'news', 'stacey', 'dash', 'group', 'director', 'hillary', 'know', 'people', 'president', 'new', 'house', '2016', 'time', 'mcmahon', 'vince', 'think', 'gave', 'paid', 'protest', 'rally', 'horner', 'protesters', 'actual', 'way', 'interview', 'clinton', 'website', 'information', 'story', 'debunking', 'thing', 'response', 'marines', 'war', 'states', 'james', 'comeys', 'comey', 'foundation', 'financial', 'peter', 'lockheed', 'martin', 'records', 'hsbc', 'dla', 'piper', 'mortgage', 'graham', 'christians', 'follow', 'nuclear', 'debatenight', 'pm', '19', 'oct', 'retweets', 'likes', 'classified']
Número de Nós no Subgrafo Filtrado: 347
Nós no Subgrafo Filtrado: ['list', 'republicans', 'supporting', 'hillary', 'clinton', 'months', 'gop', 'politicians', 'administration', 'officials', 'business', 'leaders', 'donald', 'trump', 

Etapa 3: Realização de NER (Named Entity Recognition)

In [ ]:
# Função para pré-processar o texto e agrupar termos semelhantes
def pre_processar_e_agrupar(texto):
    """
        Pré-processa o texto de cada linha do DataFrame, passando as palavras para minúsculo,
        removendo números e pontuações, adicionando equivalências a certas palavras, filtrando stop words
        e transformando o texto em tokens.

        Args:
            - texto (str): qual o conjunto de dados.

        Returns:
            - entities(list[str]): lista de tokens a serem processados.
    """

    # Substituir termos semelhantes após a remoção de pontuações
    substituicoes = {
        "trump's": 'trump',
        "donald trump's": 'trump',
        "donald trumps": 'trump',
        "trumps": 'trump',
        'hillary clinton': 'clinton',
        # Adicione mais substituições conforme necessário
    }

    for termo, substituicao in substituicoes.items():
        texto = texto.replace(termo, substituicao)

    texto = re.sub(r'[^a-zA-Z0-9\s\-\']', '', texto)

    # Remover espaços extras
    texto = ' '.join(texto.split())

    return texto

# Dentro do loop para processar o texto
def processar_texto(texto):
    # Pré-processar e agrupar termos semelhantes
    texto_preprocessado = pre_processar_e_agrupar(texto)

    # Usar spaCy para obter Named Entities
    doc = nlp(texto_preprocessado)

    # Filtrar Named Entities e remover stop words
    entities = [ent.text.lower() for ent in doc.ents if ent.text.lower() not in STOP_WORDS]

    # Adicionar atributo ner ao DataFrame
    df_fake_part.loc[df_fake_part['Texto'] == texto, 'ner'] = ', '.join(entities)

    return entities

def create_graph_NER(df:pd.DataFrame, news: str)->None:
    """
        Gera um arquivo .graphml contendo a rede de palavras presentes nas 10 primeiras
            notícias do dataset e suas respectivas classes NER.

        Args:
            - df (pd.DataFrame): DataFrame contendo uma coluna com o texto de cada notícia.
            - news (str): qual o conjunto de dados esperados para compor o nome do arquivo
                gerado.

        Returns:
            - None.
    """
    # Criar um dicionário para contar a frequência das entidades
    frequencia_entidades = {}
    for _, row in df.iterrows():
        for entidade in row['NER']:
            frequencia_entidades[entidade] = frequencia_entidades.get(entidade, 0) + 1

    G = nx.DiGraph()

    # Adicione nós e arestas com base nas Named Entities
    for _, row in df.iterrows():
        for entidade in row['NER']:
            if frequencia_entidades[entidade] >= FREQUENCIA_MINIMA:
                G.add_node(entidade, label=entidade)
                for other_entidade in row['NER']:
                    if (
                        entidade != other_entidade
                        and frequencia_entidades[other_entidade] >= FREQUENCIA_MINIMA
                        and not G.has_edge(entidade, other_entidade)
                    ):
                        G.add_edge(entidade, other_entidade)

    # Filtrar nós com base no grau usando in_degree() para grafos direcionados
    nodes_filtrados = [node for node, in_degree in G.in_degree() if in_degree >= GRAU_MINIMO]

    # Imprimir os nós filtrados
    print("Número de Nós no Subgrafo Filtrado:", len(nodes_filtrados))
    print("Nós no Subgrafo Filtrado:", nodes_filtrados)

    # Criar subgrafo apenas com os nós filtrados
    G_subgrafo = G.subgraph(nodes_filtrados)

    graph_name = 'grafo_spacy_'+news+'_filtrado_NER'
    # Salvar o grafo em formato GraphML
    nx.write_graphml(G_subgrafo, graph_name+".graphml")

# Aplicar as funções aos DataFrames
df_fake_part = df_fake.copy()[0:10].drop(axis=1, columns=['Label'])
df_fake_part['NER'] = df_fake_part['Texto'].apply(processar_texto)

df_real_part = df_real.copy()[10:20].drop(axis=1, columns=['Label'])
df_real_part['NER'] = df_real_part['Texto'].apply(processar_texto)

create_graph_NER(df_fake_part, 'fake')
create_graph_NER(df_real_part, 'real')

Número de Nós no Subgrafo Filtrado: 21
Nós no Subgrafo Filtrado: ['donald trump', 'republican', 'americans', 'trump', 'trumps', 'muslims', 'the united states', 'america', 'mexico', 'mexican', 'hillary clinton', 'american', 'clinton', 'johnson', 'florida', 'fbi', 'july', 'new york', 'epstein', 'epsteins', 'ford']


In [ ]:
# Função para pré-processar o texto e agrupar termos semelhantes
def pre_processar_e_agrupar(texto):
    """
        Pré-processa o texto de cada linha do DataFrame, passando as palavras para minúsculo,
        removendo números e pontuações, adicionando equivalências a certas palavras, filtrando stop words
        e transformando o texto em tokens.

        Args:
            - texto (str): qual o conjunto de dados.

        Returns:
            - entities(list[str]): lista de tokens a serem processados.
    """
    texto = texto.lower()
    texto = ' '.join(texto.split())

    substituicoes = {
        "trump's": 'trump',
        "donald trump's": 'trump',
        "donald trumps": 'trump',
        "trumps": 'trump',
        'hillary clinton': 'clinton',
    }

    texto = re.sub(r'[^A-Za-z\s]', '', texto)

    for termo, substituicao in substituicoes.items():
        texto = texto.replace(termo, substituicao)

    return texto

def processar_texto(texto):
    texto = pre_processar_e_agrupar(texto)
    doc = nlp(texto)
    entities = [(ent.text.lower(), ent.label_) for ent in doc.ents if ent.text.lower() not in STOP_WORDS]
    return entities

def create_graph_NER(df:pd.DataFrame, news: str)->None:
    """
        Gera um arquivo .graphml contendo a rede de palavras presentes nas 10 primeiras
            notícias do dataset e suas respectivas classes NER.

        Args:
            - df (pd.DataFrame): DataFrame contendo uma coluna com o texto de cada notícia.
            - news (str): qual o conjunto de dados esperados para compor o nome do arquivo
                gerado.

        Returns:
            - None.
    """
    # Criação do Grafo (direcionado)
    G = nx.DiGraph()

    # Adicione nós e arestas com base nas Named Entities
    for _, row in df.iterrows():
        for entidade, classe in processar_texto(row['Texto']):
            if entidade not in G.nodes:
                G.add_node(entidade, ner=classe)  # 'ner' agora é a classe do NER
            else:
                # Atualizar a classe se necessário
                G.nodes[entidade]['ner'] = classe

            for other_entidade, other_classe in processar_texto(row['Texto']):
                if (
                    entidade != other_entidade
                    and not G.has_edge(entidade, other_entidade)
                ):
                    G.add_edge(entidade, other_entidade)

                    # Imprimir as tags NER
                    #print(f"Edge: {entidade} ({classe}) - {other_entidade} ({other_classe})")

    # Filtrar nós com base no grau usando in_degree() para grafos direcionados
    nodes_filtrados = [node for node, in_degree in G.in_degree() if in_degree >= GRAU_MINIMO]

    # Imprima os nós filtrados
    print("Número de Nós no Subgrafo Filtrado:", len(nodes_filtrados))
    print("Nós no Subgrafo Filtrado:", nodes_filtrados)

    # Criar subgrafo apenas com os nós filtrados
    G_subgrafo = G.subgraph(nodes_filtrados)

    graph_name = 'grafo_spacy_'+news+'_filtrado_NER'
    # Salvar o grafo em formato GraphML
    nx.write_graphml(G_subgrafo, graph_name+".graphml")


# Aplicar as funções aos DataFrames
df_fake_part = df_fake.copy()[0:10].drop(axis=1, columns=['Label'])
df_fake_part['NER'] = df_fake_part['Texto'].apply(processar_texto)

df_real_part = df_real.copy()[10:20].drop(axis=1, columns=['Label'])
df_real_part['NER'] = df_real_part['Texto'].apply(processar_texto)

create_graph_NER(df_fake_part, 'fake')
create_graph_NER(df_real_part, 'real')

Número de Nós no Subgrafo Filtrado: 204
Nós no Subgrafo Filtrado: ['republican', 'sunday', 'maryland', 'americans', 'six months', 'half', 'chris', 'new jersey', 'ben carson', 'february', 'donald j trump', 'this week', 'election year', 'months', 'this year', 'years', 'david eisenhower', 'europe', 'democrat', 'adlai stevenson', 'washington', 'muslims', 'the iraq war', 'the united states', 'islamic', 'eleven million', 'america', 'japan', 'south korea', 'islamist', 'muslim', 'thousands', 'mexico', 'the year', 'nato', 'eight years', 'republicans', 'second', 'congress', 'seven', 'mexican', 'john f kennedy clinton', 'the republican party', 'this weeks', 'the past three decades', 'only months ago', 'democratic', 'clinton', 'donald trump', 'american', 'cbs', 'ac', 'the first three months', 'pbs', 'paul', 'steve bannon', 'a day every day', 'the past  years', 'hours', 'judy woodruff', 'atlanta', 'french', 'italians', 'today', 'kim', 'the next four years', 'brillat de', 'iraq', 'last month', 'fift

In [ ]:
from nltk import ne_chunk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
import pandas as pd

# Certifique-se de ter nltk e punkt instalados
# Você pode instalar usando: pip install nltk
# E também pode precisar baixar os recursos do punkt:
# import nltk
# nltk.download('punkt')

# Suponha que você já tenha uma coluna 'PoS_Tags' no DataFrame df_concat
# Se não tiver, você precisa adicionar essa coluna antes de executar o código a seguir

# Função para aplicar NER a uma lista de PoS tags
def apply_ner(pos_tags):
    # Converta as tags PoS em formato nltk para entrada do ne_chunk
    words = [word for word, pos_tag in pos_tags]
    pos_tags_nltk = [(word, nltk_pos_tag) for (word, pos_tag), nltk_pos_tag in zip(pos_tags, nltk.pos_tag(words))]

    # Executar NER nas tags de PoS
    ner_tags = ne_chunk(pos_tags_nltk)

    return ner_tags

# Aplicar a função a cada linha da coluna 'PoS_Tags'
df_concat['NER_Tags'] = df_concat['PoS_Tags'].apply(apply_ner)

# Exibir o DataFrame resultante
print(df_concat[['Texto', 'NER_Tags']])


AttributeError: 'tuple' object has no attribute 'startswith'

In [ ]:
from nltk import ne_chunk

# Executar NER nas sentenças tokenizadas
ner_tags = ne_chunk(pos_tags)

#print("NER Tags:", ner_tags)


Etapa 4: Criação de Grafos com NetworkX

In [ ]:
df

,Texto,Label,Previsoes
0,The Internet is buzzing today after white supr...,1,"[0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,TRUMP TOWER With his poll numbers among black...,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Donald Trump is looking to assemble a strong t...,1,"[0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"SHES SEXY, SMART, SOPHISTICATED, AND SHES INTO...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"PHOENIX A.Z. (AP) For months now, rumors have...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
96,The Upshots elections model suggests that Hill...,0,"[0, 0, 0, 6, 0, 6, 0, 0, 0, 0, 4, 4, 0, 0, 0, ..."
97,"I dont know about you, but Im getting tired of...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
98,When Donald Trump began his improbable run for...,0,"[0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
99,An anonymous Jane Doe filed a federal lawsuit ...,0,"[0, 0, 0, 0, 4, 4, 4, 0, 0, 0, 0, 0, 2, 2, 0, ..."


In [ ]:
import networkx as nx
import json

# Converter listas em strings usando JSON
df['Previsoes'] = df['Previsoes'].apply(json.dumps)

# Criar um grafo para PoS Tagging
grafo_pos = nx.DiGraph()

# Adicionar nós ao grafo com base nas relações PoS Tagging
for i in range(len(df)):
    palavra, pos_tag_palavra = df['Texto'][i], df['Previsoes'][i]
    grafo_pos.add_node(palavra, pos=pos_tag_palavra)

# Adicionar arestas ao grafo
for i in range(len(df) - 1):
    palavra_atual, pos_tag_atual = df['Texto'][i], df['Previsoes'][i]
    palavra_proxima, pos_tag_proxima = df['Texto'][i + 1], df['Previsoes'][i + 1]

    grafo_pos.add_edge(palavra_atual, palavra_proxima, pos_tag=pos_tag_proxima)

# Salvar o grafo em um arquivo GraphML
nx.write_graphml(grafo_pos, "grafo_posT.graphml")


In [ ]:
import networkx as nx

def criar_grafo_pos_por_tag(texto, previsoes_pos, tag_interessante='NN'):
    grafo_pos = nx.DiGraph()

    # Adicionar nós e arestas ao grafo com base nas relações PoS Tagging
    for i in range(len(previsoes_pos)):
        palavra, pos_tag_palavra = texto[i], previsoes_pos[i]

        # Adicionar nós e arestas apenas para palavras com a tag de interesse
        if pos_tag_palavra == tag_interessante:
            grafo_pos.add_node(palavra, pos=pos_tag_palavra)

            # Adicionar aresta para a próxima palavra se existir
            if i < len(previsoes_pos) - 1:
                proxima_palavra, proximo_pos_tag_palavra = texto[i + 1], previsoes_pos[i + 1]
                if proximo_pos_tag_palavra == tag_interessante:
                    grafo_pos.add_node(proxima_palavra, pos=proximo_pos_tag_palavra)
                    grafo_pos.add_edge(palavra, proxima_palavra)

    return grafo_pos

# Supondo que 'df' é o seu DataFrame
grafos_pos_dict = {}
for idx, row in df.iterrows():
    print(f"Index: {idx}, Comprimento Texto: {len(row['Texto'])}, Comprimento Previsoes: {len(row['Previsoes'])}")
    try:
        grafo_pos = criar_grafo_pos_por_tag(row['Texto'], row['Previsoes'])
        grafos_pos_dict[idx] = grafo_pos
    except ValueError as e:
        print(f"Erro no índice {idx}: {e}")

# Salvar os grafos em arquivos GraphML
for idx, grafo_pos in grafos_pos_dict.items():
    nx.write_graphml(grafo_pos, f"grafo_pos_{idx}.graphml")


Index: 0, Comprimento Texto: 1197, Comprimento Previsoes: 1671


IndexError: string index out of range

In [ ]:
# Criar um grafo para PoS Tagging
grafo_pos = nx.DiGraph()

# Adicionar nós e arestas ao grafo com base nas relações PoS Tagging
for i in range(len(pos_tags) - 1):
    palavra, pos_tag_palavra = pos_tags[i]
    proxima_palavra, proximo_pos_tag_palavra = pos_tags[i + 1]

    grafo_pos.add_node(palavra, pos=pos_tag_palavra)
    grafo_pos.add_node(proxima_palavra, pos=proximo_pos_tag_palavra)
    grafo_pos.add_edge(palavra, proxima_palavra)

nx.write_graphml(grafo_pos, "grafo_pos.graphml")


NameError: name 'pos_tags' is not defined

In [ ]:
# Criar um grafo para NER
grafo_ner = nx.DiGraph()

# Adicionar nós e arestas ao grafo com base nas relações NER
for chunk in ner_tags:
    if isinstance(chunk, Tree) and hasattr(chunk, 'label'):
        entidade = ' '.join(c[0] for c in chunk.leaves())
        grafo_ner.add_node(entidade, ner=chunk.label())
        for palavra, pos_tag in chunk.leaves():
            grafo_ner.add_edge(palavra, entidade)

nx.write_graphml(grafo_ner, "grafo_ner.graphml")


NameError: ignored

Etapa 5: Análise de Rede

In [ ]:
# Realizar análise de rede (por exemplo, centralidade, grau, etc.)
# Aqui, um exemplo de cálculo de grau dos nós
grau_nos1 = dict(grafo_pos.degree())
grau_nos2 = dict(grafo_ner.degree())
print("Grau dos nós:", grau_nos1)
print("Grau dos nós:", grau_nos2)


Etapa 6: Visualização da Rede

In [ ]:
import matplotlib.pyplot as plt

# Desenhar o grafo
posicao = nx.spring_layout(grafo_pos)  # Pode escolher outros layouts
nx.draw(grafo_pos, with_labels=True, font_size=8, font_color='black', node_size=1000, node_color='skyblue', font_weight='bold', arrowsize=10)

# Exibir a visualização
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Desenhar o grafo
posicao = nx.spring_layout(grafo_ner)  # Pode escolher outros layouts
nx.draw(grafo_ner, with_labels=True, font_size=8, font_color='black', node_size=1000, node_color='skyblue', font_weight='bold', arrowsize=10)

# Exibir a visualização
plt.show()